In [25]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [26]:
from pathlib import Path
import pandas as pd

from src.pipeline.pipeline_builder import PipelineBuilder

LOG_PATH = Path("../data/raw/csv/kaggle/Registro_de_Incidentes.csv")
assert LOG_PATH.exists(), f"Arquivo de log não encontrado: {LOG_PATH}"
LOG_PATH

PosixPath('../data/raw/csv/kaggle/Registro_de_Incidentes.csv')

In [27]:
df_events_incidents = pd.read_csv(LOG_PATH, sep=";", parse_dates=["Timestamp"])

df_events_incidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242901 entries, 0 to 242900
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Case ID                242901 non-null  object        
 1   Variant                242901 non-null  object        
 2   Priority               242901 non-null  object        
 3   Reporter               242901 non-null  object        
 4   Timestamp              242901 non-null  datetime64[ns]
 5   Event                  242901 non-null  object        
 6   Issue Type             242901 non-null  object        
 7   Resolver               146405 non-null  object        
 8   Report Channel         242901 non-null  object        
 9   Short Description      242901 non-null  object        
 10  Customer Satisfaction  242901 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 20.4+ MB


/var/folders/qy/v482m2215v59dlznrv8yyrp80000gn/T/ipykernel_92592/2280142455.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_events_incidents = pd.read_csv(LOG_PATH, sep=";", parse_dates=["Timestamp"])


In [28]:
df_events_incidents.head(10)

,Case ID,Variant,Priority,Reporter,Timestamp,Event,Issue Type,Resolver,Report Channel,Short Description,Customer Satisfaction
0,INC0001,Variant 4,Medium,Alice,2023-11-17 11:17:00,Ticket created,Performance Issue,NaN,Website,Application crash,3
1,INC0001,Variant 4,Medium,Alice,2023-11-17 13:30:00,Ticket assigned to level 1 support,Performance Issue,Sam,Website,Application crash,3
2,INC0001,Variant 4,Medium,Alice,2023-11-17 14:06:00,WIP - level 1 support,Performance Issue,Sam,Website,Application crash,3
3,INC0001,Variant 4,Medium,Alice,2023-11-17 16:52:00,Level 1 escalates to level 2 support,Performance Issue,Michael,Website,Application crash,3
4,INC0001,Variant 4,Medium,Alice,2023-11-17 18:07:00,WIP - level 2 support,Performance Issue,Emma,Website,Application crash,3
5,INC0001,Variant 4,Medium,Alice,2023-11-17 20:10:00,Ticket solved by level 2 support,Performance Issue,Sarah,Website,Application crash,3
6,INC0001,Variant 4,Medium,Alice,2023-11-17 21:28:00,Ticket closed,Performance Issue,NaN,Website,Application crash,3
7,INC0002,Variant 3,High,Charlie,2023-08-15 07:08:00,Ticket created,Performance Issue,NaN,App,Data loss issue,3
8,INC0002,Variant 3,High,Charlie,2023-08-15 08:29:00,Ticket assigned to level 1 support,Performance Issue,Sam,App,Data loss issue,3
9,INC0002,Variant 3,High,Charlie,2023-08-15 08:39:00,WIP - level 1 support,Performance Issue,Sam,App,Data loss issue,3


In [29]:
df_events_incidents.describe()

,Timestamp,Customer Satisfaction
count,242901,242901.000000
mean,2023-07-01 18:31:09.225734400,3.233828
min,2023-01-01 00:07:00,1.000000
25%,2023-04-01 07:23:00,3.000000
50%,2023-07-01 10:22:00,3.000000
75%,2023-09-30 19:23:00,4.000000
max,2024-01-02 07:44:00,5.000000
std,NaN,1.161075


In [30]:
# drop time stamp for encoding
df_events_incidents_no_timestamp = df_events_incidents.drop(columns=["Timestamp"])

In [31]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

df = df_events_incidents_no_timestamp.copy()

categorical_cols = [
    "Case ID",
    "Variant",
    "Priority",
    "Reporter",
    "Event",
    "Issue Type",
    "Resolver",
    "Report Channel",
    "Short Description",
]

numeric_cols = ["Customer Satisfaction"]


preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)

X_encoded = preprocess.fit_transform(df)

In [33]:
print(X_encoded)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2429010 stored elements and shape (242901, 31657)>
  Coords	Values
  (0, 0)	1.0
  (0, 31595)	1.0
  (0, 31603)	1.0
  (0, 31604)	1.0
  (0, 31617)	1.0
  (0, 31631)	1.0
  (0, 31643)	1.0
  (0, 31647)	1.0
  (0, 31648)	1.0
  (0, 31656)	3.0
  (1, 0)	1.0
  (1, 31595)	1.0
  (1, 31603)	1.0
  (1, 31604)	1.0
  (1, 31614)	1.0
  (1, 31631)	1.0
  (1, 31640)	1.0
  (1, 31647)	1.0
  (1, 31648)	1.0
  (1, 31656)	3.0
  (2, 0)	1.0
  (2, 31595)	1.0
  (2, 31603)	1.0
  (2, 31604)	1.0
  (2, 31624)	1.0
  :	:
  (242898, 31631)	1.0
  (242898, 31640)	1.0
  (242898, 31645)	1.0
  (242898, 31649)	1.0
  (242898, 31656)	4.0
  (242899, 24746)	1.0
  (242899, 31599)	1.0
  (242899, 31603)	1.0
  (242899, 31605)	1.0
  (242899, 31610)	1.0
  (242899, 31631)	1.0
  (242899, 31643)	1.0
  (242899, 31645)	1.0
  (242899, 31649)	1.0
  (242899, 31656)	4.0
  (242900, 24746)	1.0
  (242900, 31599)	1.0
  (242900, 31603)	1.0
  (242900, 31605)	1.0
  (242900, 31616)	1.0
  (242900, 3